# How can we have lazy class attributes?

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from types import MethodType
from time import sleep
from abc import ABC, ABCMeta, abstractmethod
from functools import wraps
import inspect


def compute(obj, s, time=1):
    print(f"Computing {s} of {obj} ...", end="")
    sleep(time)
    print("DONE!")
    return "Phew, that was a lot of work!"

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(
            "Property.__init__:",
            f"{self=}",
            f"{fget=}",
            f"{fset=}",
            f"{fdel=}",
            f"{doc=}",
            sep="\n\t",
        )
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        self.decorated_obj = None

    def __call__(self, *args, **kwargs):
        return self.fget.__call__(*args, **kwargs)

    def __get__(self, obj, objtype=None):
        print("Property.__get__:", f"{self=}", f"{obj=}", f"{objtype=}", sep="\n\t")
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        if obj is None:
            result = self
        else:
            self.decorated_obj = obj
            result = self.fget(obj)
        print(f">>> Property.__get__ ", f"{(obj is None)=}", f"{result=}", sep="\n\t")
        return result

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class ClassMethodX(property):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, *args, **kwargs):
        print(
            f"ClassMethod.__init__:", f"{self=}", f"{args=}", f"{kwargs=}", sep="\n\t"
        )
        super().__init__()

    def __call__(self, *args, **kwargs):
        return self.fget.__call__(*args, **kwargs)

    def __get__(self, obj, cls=None):
        print(f"ClassMethod.__get__:", f"{self=}", f"{obj=}", f"{cls=}", sep="\n\t")
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.fget), "__get__"):
            # result = self.f.__get__(cls)
            result = MethodType(self.fget.__get__, cls).__call__()
        else:
            result = MethodType(self.fget, cls)
        print(
            f">>> ClassMethod.__get__:",
            f"{hasattr(type(self.f), '__get__')=}",
            f"{result=}",
            sep="\n\t",
        )
        return result

In [ ]:
class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        print(f"ClassMethod.__init__:", f"{self=}", f"{f=}", sep="\n\t")
        self.f = f

    def __call__(self, *args, **kwargs):
        return self.f.__call__(*args, **kwargs)

    def __get__(self, obj, cls=None):
        print(f"ClassMethod.__get__:", f"{self=}", f"{obj=}", f"{cls=}", sep="\n\t")
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.f), "__get__"):
            # result = self.f.__get__(cls)
            result = MethodType(self.f.__get__, cls).__call__()
        else:
            result = MethodType(self.f, cls)
        print(
            f">>> ClassMethod.__get__:",
            f"{hasattr(type(self.f), '__get__')=}",
            f"{result=}",
            sep="\n\t",
        )
        return result

In [ ]:
class ClassPropertyMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(
            "ClassPropertyMethod.__init__:",
            f"{self=}",
            f"{fget=}",
            f"{fset=}",
            f"{fdel=}",
            f"{doc=}",
            sep="\n\t",
        )
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, cls=None):
        print(
            f"ClassPropertyMethod.__get__:",
            f"{self=}",
            f"{obj=}",
            f"{cls=}",
            sep="\n\t",
        )
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.fget), "__get__"):
            # result = self.f.__get__(cls)
            result = MethodType(self.fget.__get__, cls).__call__()
        else:
            result = MethodType(self.fget, cls)
        print(
            f">>> ClassPropertyMethod.__get__:",
            f"{hasattr(type(self.fget), '__get__')=}",
            f"{result=}",
            sep="\n\t",
        )
        return result

    def __set__(self, obj, value):
        if self.fset.fset is None:
            raise AttributeError("can't set attribute")
        self.fset.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class ClassProperty(property):
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(
            "ClassProperty.__init__:",
            f"{self=}",
            f"{fget=}",
            f"{fset=}",
            f"{fdel=}",
            f"{doc=}",
            sep="\n\t",
        )
        # super().__init__(ClassMethod(fget), fset, fdel, doc)
        # self.fget = fget #if fget is None else property(fget) if fget is not None else fget
        # self.fset = fset #if fset is None else property(fset) if fset is not None else fset
        # self.fdel = fdel #if fdel is None else property(fdel) if fdel is not None else fdel
        # if doc is None and fget is not None:
        # doc = fget.__doc__
        # self.__doc__ = doc

    # def __get__(self, obj, cls=None):
    #     print(f"ClassProperty.__get__:", f"{self=}", f"{obj=}", f"{cls=}", sep="\n\t")
    #     return self.fget(obj)
    # if cls is None:
    #     cls = type(obj)
    # if hasattr(type(self.fget), "__get__"):
    #     # result = self.f.__get__(cls)
    #     result = MethodType(self.fget.__get__, cls).__call__()
    # else:
    #     result = MethodType(self.fget, cls)
    # print(
    #     f">>> ClassProperty.__get__:",
    #     f"{hasattr(type(self.fget), '__get__')=}",
    #     f"{result=}",
    #     sep="\n\t",
    # )
    # return property(result).__get__(obj)

In [ ]:
class Testit:
    @classmethod
    @property
    def clsprop(cls):
        """Real Class-Property"""
        return f"My name is {cls.__name__}"

    @property
    @classmethod
    def wrongclsprop(cls):
        """Wrong-order Class-Property"""
        return f"My name is {cls.__name__}"

    @Property
    @ClassMethod
    def mywrongclsprop(cls):
        """Wrong-order Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassMethod
    @Property
    def myclsprop(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassPropertyMethod
    @Property
    def myclsprop2(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassProperty
    def myclsprop3(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"

    @property
    def prop(self):
        return 42


display(Testit.__dict__)
Testit.myclsprop, Testit.myclsprop2, Testit.myclsprop3

In [ ]:
Testit.mywrongclsprop

In [ ]:
help(Testit)

In [ ]:
inspect.signature(classmethod.__init__)

In [ ]:
Testit.__dict__["myclsprop3"].__dir__()

In [ ]:
inspect.signature(Testit.myclsprop3.__get__)

In [ ]:
Testit.myclsprop, Testit.myclsprop2, Testit.myclsprop3

In [ ]:
Testit.__dict__["myclsprop3"].fget

In [ ]:
Testit.__dict__["myclsprop2"].__isabstractmethod__ = False

In [ ]:
Testit.myclsprop3

In [ ]:
inspect.signature(wraps)

## Progress

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(
            "Property.__init__:",
            f"{self=}",
            f"{fget=}",
            f"{fset=}",
            f"{fdel=}",
            f"{doc=}",
            sep="\n\t",
        )
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        # self.decorated_obj = None

    #     def __call__(self, *args, **kwargs):
    #         return self.fget.__call__(*args, **kwargs)

    def __get__(self, obj, objtype=None):
        print("Property.__get__:", f"{self=}", f"{obj=}", f"{objtype=}", sep="\n\t")
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        if obj is None:
            result = self
        else:
            # self.decorated_obj = obj
            result = self.fget(obj)
        print(f">>> Property.__get__ ", f"{(obj is None)=}", f"{result=}", sep="\n\t")
        return result

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class ClassMethod(property):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(
            "ClassMethod.__init__:",
            f"{self=}",
            f"{fget=}",
            f"{fset=}",
            f"{fdel=}",
            f"{doc=}",
            sep="\n\t",
        )
        super().__init__(fget, fset, fdel, doc)

        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    # def __call__(self, *args, **kwargs):
    #     return self.f.__call__(*args, **kwargs)

    def __get__(self, obj, cls=None):
        print(f"ClassMethod.__get__:", f"{self=}", f"{obj=}", f"{cls=}", sep="\n\t")
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.fget), "__get__"):
            # result = self.f.__get__(cls)
            print("BRANCH 1")
            result = MethodType(self.fget.__get__, cls).__call__()
        else:
            print("BRANCH 2")
            result = MethodType(self.fget, cls)
        print(
            f">>> ClassMethod.__get__:",
            f"{hasattr(type(self.fget), '__get__')=}",
            f"{result=}",
            sep="\n\t",
        )
        return result

In [ ]:
class LazyAttribute(property):
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(
            "ClassMethod.__init__:",
            f"{self=}",
            f"{fget=}",
            f"{fset=}",
            f"{fdel=}",
            f"{doc=}",
            sep="\n\t",
        )

        super().__init__(fget, fset, fdel, doc)

        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, cls=None):
        print(f"ClassMethod.__get__:", f"{self=}", f"{obj=}", f"{cls=}", sep="\n\t")
        if cls is None:
            cls = type(obj)

        result = MethodType(property(self.fget).__get__, cls).__call__()
        # if hasattr(type(self.fget), "__get__"):
        #     # result = self.f.__get__(cls
        #     result = MethodType(self.fget.__get__, cls).__call__()
        # else:
        #     result = MethodType(self.fget, cls)
        print(
            f">>> ClassMethod.__get__:",
            f"{hasattr(type(self.fget), '__get__')=}",
            f"{result=}",
            sep="\n\t",
        )
        return result

In [ ]:
class ClassProperty(property):
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        fget = property(fget)
        super().__init__(fget, fset, fdel, doc)
        self.__doc__ = fget.__doc__ if doc is None else doc

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return MethodType(self.fget.__get__, cls).__call__()

    def __set__(self, obj, value):
        print(f"__SET__ called", f"{obj=}", f"{value=}", f"{self.fset=}", sep="\t\n")
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class Testit:
    @ClassProperty
    def myclsprop(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"


display(Testit.__dict__)
display(Testit.__dict__["myclsprop"].__dir__())
display(Testit.myclsprop)

In [ ]:
Testit.__dict__["myclsprop"].fset is None

In [ ]:
help(Testit)

In [ ]:
Testit.myclsprop = 2

In [ ]:
Testit.myclsprop = 2

In [ ]:
Testit.__dict__

In [ ]:
class ClassMethod(property):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, func):
        self.func = classmethod(func)

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.func), "__get__"):
            print("Branch 1")
            return self.func.__get__(cls)
        return MethodType(self.func, cls)

In [ ]:
class Testit:
    @ClassMethod
    @property
    def myclsprop(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"


display(Testit.__dict__)
Testit.myclsprop

In [ ]:
help(Testit)

In [ ]:
Testit.__dict__["myclsprop"]

In [ ]:
from time import sleep
from abc import ABC, ABCMeta, abstractmethod


class MyMetaClass(ABCMeta):
    @classmethod
    @property
    def expensive_metaclass_property(cls):
        """This may take a while to compute!"""
        print("computing metaclass property")
        sleep(3)
        return "Phew, that was a lot of work!"


class MyBaseClass(ABC, metaclass=MyMetaClass):
    @classmethod
    @property
    def expensive_class_property(cls):
        """This may take a while to compute!"""
        print("computing class property ..")
        sleep(3)
        return "Phew, that was a lot of work!"

    @property
    def expensive_instance_property(self):
        """This may take a while to compute!"""
        print("computing instance property ...")
        sleep(3)
        return "Phew, that was a lot of work!"


class MyClass(MyBaseClass):
    """Some subclass of MyBaseClass"""

In [ ]:
help(MyClass)

In [ ]:
MyBaseClass.expensive_class_property = 2
MyBaseClass().expensive_instance_property = 2
# MyBaseClass.__dict__

In [ ]:
instance = MyBaseClass()
instance.expensive_class_property = 2
instance.expensive_instance_property = 2

In [ ]:
import math


class TrigConst:
    def __init__(self, const=math.pi):
        self.const = const

    def __get__(self, obj, objtype=None):
        return self.const

    def __set__(self, obj, value):
        print(f"__set__ called", f"{obj=}")
        self.const = value


class Trig:
    const = TrigConst()


class PatchedSetattr(type):
    def __setattr__(cls, key, value):
        if hasattr(cls, key):
            obj = cls.__dict__[key]
            if hasattr(obj, "__set__"):
                obj.__set__(cls, value)
        else:
            super().__setattr__(key, value)


class PatchedTrig(metaclass=PatchedSetattr):
    const = TrigConst()

In [ ]:
inst = Trig()
print(inst.const, type(inst.__class__.__dict__["const"]))
inst.const = math.tau
print(inst.const, type(inst.__class__.__dict__["const"]))

In [ ]:
cls = Trig
print(cls.const, type(cls.__dict__["const"]))
cls.const = math.tau
print(cls.const, type(cls.__dict__["const"]))

In [ ]:
inst = PatchedTrig()
print(inst.const, type(inst.__class__.__dict__["const"]))
inst.const = math.tau
print(inst.const, type(inst.__class__.__dict__["const"]))

In [ ]:
cls = PatchedTrig
print(cls.const, type(cls.__dict__["const"]))
cls.const = math.tau
print(cls.const, type(cls.__dict__["const"]))

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)


class LoggedAgeAccess:
    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info("Accessing %r giving %r", "age", value)
        return value

    def __set__(self, obj, value):
        logging.info("Updating %r to %r", "age", value)
        obj._age = value


class Person:

    age = LoggedAgeAccess()  # Descriptor instance

    def __init__(self, name, age):
        self.name = name  # Regular instance attribute
        self.age = age  # Calls __set__()

    def birthday(self):
        self.age += 1  # Calls both __get__() and __set__()

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)


class LoggedAgeAccess:
    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info("Accessing %r giving %r", "age", value)
        return value

    def __set__(self, obj, value):
        logging.info("Updating %r to %r", "age", value)
        obj._age = value


class Person:

    age = LoggedAgeAccess()  # Descriptor instance

    def __init__(self, name, age):
        self.name = name  # Regular instance attribute
        self.age = age  # Calls __set__()

    def birthday(self):
        self.age += 1  # Calls both __get__() and __set__()

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)


class LoggedAgeAccess:
    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info("Accessing %r giving %r", "age", value)
        return value

    def __set__(self, obj, value):
        logging.info("Updating %r to %r", "age", value)
        obj._age = value


class Person:

    age = LoggedAgeAccess()  # Descriptor instance

    def __init__(self, name, age):
        self.name = name  # Regular instance attribute
        self.age = age  # Calls __set__()

    def birthday(self):
        self.age += 1  # Calls both __get__() and __set__()